In [42]:
import pandas as pd
import sqlite3

## 1

In [43]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## 2

In [44]:
pd.io.sql.read_sql(
    """
    PRAGMA table_info(test);
    """,
    con=conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TEXT,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## 3

In [45]:
pd.io.sql.read_sql(
    """
    SELECT * FROM test LIMIT 10;
    """,
    con=conn
)

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## 4

In [46]:
pd.io.sql.read_sql(
    """
    SELECT * FROM deadlines;
    """,
    con=conn
)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [47]:
df_min = pd.io.sql.read_sql(
    """
    SELECT t.uid, ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2) AS delta
    FROM test AS t LEFT JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname <> "project1"
    ORDER BY ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2)
    LIMIT 1;
    """, 
    con=conn
)

In [48]:
df_min

,uid,delta
0,user_25,2.87


## 5

In [49]:
df_max = pd.io.sql.read_sql(
    """
    SELECT t.uid, ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2) AS delta
    FROM test AS t LEFT JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname <> "project1"
    ORDER BY ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2) DESC
    LIMIT 1;
    """, 
    con=conn
)

In [50]:
df_max

,uid,delta
0,user_30,202.38


## 6

In [51]:
df_avg = pd.io.sql.read_sql(
    """
    SELECT AVG(ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2)) AS avg
    FROM test AS t LEFT JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname <> "project1";
    """, 
    con=conn
)

In [52]:
df_avg

,avg
0,89.687917


## 7

In [60]:
pd.io.sql.read_sql(
    """
    SELECT uid, COUNT(*)
    FROM pageviews
    GROUP BY uid
    """,
    con=conn
)

,uid,COUNT(*)
0,admin_0,3
1,admin_1,71
2,admin_2,1
3,admin_3,18
4,user_1,28
5,user_10,89
6,user_14,143
7,user_17,47
8,user_18,3
9,user_19,16


In [61]:
views_diff = pd.io.sql.read_sql(
    """
    SELECT t.uid, AVG(ROUND((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24, 2)) AS avg_diff, p.cnt AS pageviews
    FROM test AS t LEFT JOIN deadlines AS d ON t.labname = d.labs 
                   LEFT JOIN (
                        SELECT uid, COUNT(*) AS cnt
                        FROM pageviews 
                        GROUP BY uid
                   ) AS p ON t.uid = p.uid
    WHERE t.labname <> "project_1"
    GROUP BY t.uid
    """,
    con=conn
)

In [64]:
views_diff

,uid,avg_diff,pageviews
0,user_1,62.776667,28
1,user_10,79.333333,89
2,user_14,186.277500,143
3,user_17,166.513333,47
4,user_18,38.232500,3
5,user_19,91.842000,16
6,user_21,107.420000,10
7,user_25,112.035000,179
8,user_28,89.875000,149
9,user_3,124.608333,317


In [67]:
views_diff[['avg_diff', 'pageviews']].corr()

,avg_diff,pageviews
avg_diff,1.000000,0.069118
pageviews,0.069118,1.000000


## 8

In [68]:
conn.close()